In [54]:
import numpy as np

In [55]:
import tensorflow as tf 

In [56]:
def huber_fn(y_true,y_pred):
    error = y_true,y_pred
    is_small_error = tf.abs(error)<1
    squared_loss = tf.square(error)/2
    linear_loss = tf.abs(error)-0.5
    return tf.where(is_small_error,squared_loss,linear_loss)

In [57]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

input_shape = X_train.shape[1:]

tf.keras.utils.set_random_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal",
                          input_shape=input_shape),
    tf.keras.layers.Dense(1),
])

/opt/anaconda3/envs/Data_pytorcch/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [58]:
model.compile(loss=huber_fn,optimizer='nadam')
model.fit(X_train_scaled,y_train,validation_data=(X_valid_scaled,y_valid),epochs=10)

Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 806us/step - loss: 0.8762 - val_loss: 0.8128
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step - loss: 0.8093 - val_loss: 0.7901
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step - loss: 0.8062 - val_loss: 0.7773
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 418us/step - loss: 0.8051 - val_loss: 0.7743
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step - loss: 0.8046 - val_loss: 0.7789
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - loss: 0.8044 - val_loss: 0.7741
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step - loss: 0.8042 - val_loss: 0.7778
Epoch 8/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 408us/step - loss: 0.8041 - val_loss: 0.7745
Epoch 9/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 424us/step - loss: 0.8041 - val_loss: 0.7773
Epoch 10/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 421us/step - loss: 0.8039 - val_loss: 0.7750


In [59]:
from tensorflow import keras
model.save("custom_model.keras")

In [60]:
model= keras.models.load_model("./custom_model.keras",custom_objects={"huber_fn":huber_fn})

In [61]:
model.fit(X_train_scaled,y_train,epochs=10,validation_data=(X_valid_scaled,y_valid))

Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step - loss: 0.8038 - val_loss: 0.7768
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 400us/step - loss: 0.8037 - val_loss: 0.7753
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step - loss: 0.8037 - val_loss: 0.7765
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 396us/step - loss: 0.8036 - val_loss: 0.7765
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step - loss: 0.8037 - val_loss: 0.7761
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step - loss: 0.8036 - val_loss: 0.7764
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step - loss: 0.8036 - val_loss: 0.7759
Epoch 8/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 470us/step - loss: 0.8036 - val_loss: 0.7762
Epoch 9/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 525us/step - loss: 0.8036 - val_loss: 0.7763
Epoch 10/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 439us/step - loss: 0.8035 - val_loss: 0.7762


In [62]:
def create_huber(threshold=1.0):
    def huber_fn(y_true,y_pred):
        error = y_true-y_pred
        is_small_error = tf.abs(error)<threshold
        sqaured_loss= tf.square(error)/2
        linear_loss  = threshold*tf.abs(error)-threshold**2/2
        return tf.where(is_small_error,sqaured_loss,linear_loss)
    return huber_fn



In [63]:
model.compile(loss=create_huber(2.0),optimizer="nadam")

In [64]:
model.save("custom_model_with_threshold.keras")

In [65]:
model= keras.models.load_model("custom_model_with_threshold.keras",custom_objects={"huber_fn":create_huber(2.0)})

/opt/anaconda3/envs/Data_pytorcch/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 11 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [66]:
class HuberClass(keras.losses.Loss):
    def __init__(self, threshold=2.0,**kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self,y_true,y_pred):
        error  = y_true-y_pred
        is_small_error= tf.abs(error)<self.threshold
        squared_loss = tf.square(error)/2
        linear_loss = self.threshold*tf.abs(error)-self.threshold**2/2
        return tf.where(is_small_error,squared_loss,linear_loss)
    def get_config(self):
        base_config=super().get_config()
        return {**base_config,"threshold":self.threshold}

create a new model

In [67]:
tf.keras.utils.set_random_seed(42)
model = keras.models.Sequential([
    tf.keras.layers.Dense(30,activation="relu",kernel_initializer="he_normal",input_shape=input_shape),
    tf.keras.layers.Dense(1)
])
model

<Sequential name=sequential_6, built=True>

In [68]:
model.compile(loss=HuberClass(2.),optimizer="nadam",metrics=["mae"])

In [69]:
model.fit(X_train_scaled,y_train,epochs=10,validation_data=(X_valid_scaled,y_valid))

Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 862us/step - loss: 1.0512 - mae: 1.1438 - val_loss: 0.5086 - val_mae: 0.6718
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 658us/step - loss: 0.3170 - mae: 0.5816 - val_loss: 0.3527 - val_mae: 0.5571
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - loss: 0.2611 - mae: 0.5243 - val_loss: 0.2689 - val_mae: 0.4982
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 848us/step - loss: 0.2328 - mae: 0.4929 - val_loss: 0.2207 - val_mae: 0.4657
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 496us/step - loss: 0.2178 - mae: 0.4754 - val_loss: 0.1899 - val_mae: 0.4453
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step - loss: 0.2093 - mae: 0.4654 - val_loss: 0.1996 - val_mae: 0.4471
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 462us/step - loss: 0.2036 - mae: 0.4589 - val_loss: 0.2243 - val_mae: 0.4549
Epoch 8/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 462us/step - loss: 0.1995 - mae: 0.4528 - val_loss: 0.1774 - val_mae: 0.4297
Epoch 9/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 

In [70]:
model.save("custom_model_with_loss_function_class.keras")

In [71]:
model= keras.models.load_model("./custom_model_with_loss_function_class.keras",custom_objects={"HuberClass":HuberClass})

In [72]:
model.fit(X_train_scaled,y_train,validation_data=(X_valid,y_valid),epochs=2)

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 808us/step - loss: 0.1904 - mae: 0.4411 - val_loss: 429.6693 - val_mae: 215.8347
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 455us/step - loss: 0.1878 - mae: 0.4383 - val_loss: 437.0641 - val_mae: 219.5320


Custom Activations , Initializers , Regularizers ,and Constraints

In [73]:
def my_softpluz(z):
    return tf.math.log(tf.exp(z)+1.0)


In [74]:
def my_glorot_initializer(shape,dtype=tf.float32):
    stddev = tf.sqrt(2./(shape[0]+shape[1]))
    return tf.random.normal(shape,stddev=stddev,dtype=dtype)

In [75]:
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01*weights))

In [76]:
def my_positive_weights(weights):
    return tf.where(weights<0.,tf.zeros_like(weights),weights)

In [77]:
layer = keras.layers.Dense(30,activation=my_softpluz,kernel_initializer=my_glorot_initializer,kernel_regularizer=my_l1_regularizer,kernel_constraint=my_positive_weights)

In [78]:
class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self,factor):
        self.factor = factor
    def __call__(self,weights):
        return tf.reduce_sum(tf.abs(self.factor*weights))
    def get_config(self):
        return {"factor":self.factor}

In [79]:
tf.keras.utils.set_random_seed(42)
model = keras.models.Sequential([
    tf.keras.layers.Dense(30,activation="relu",kernel_initializer="he_normal",input_shape=input_shape),
    tf.keras.layers.Dense(1)
    
])

In [80]:
model.compile(loss="mse",optimizer="nadam",metrics=[create_huber(2.0)])

In [81]:
model.fit(X_train_scaled,y_train,epochs=10)

Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 375us/step - huber_fn: 1.0833 - loss: 2.6041
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step - huber_fn: 0.3403 - loss: 0.7644
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step - huber_fn: 0.2811 - loss: 0.6060
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - huber_fn: 0.2461 - loss: 0.5150
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 329us/step - huber_fn: 0.2273 - loss: 0.4674
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 327us/step - huber_fn: 0.2166 - loss: 0.4419
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 302us/step - huber_fn: 0.2097 - loss: 0.4269
Epoch 8/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 321us/step - huber_fn: 0.2048 - loss: 0.4167
Epoch 9/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step - huber_fn: 0.2010 - loss: 0.4091
Epoch 10/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 316us/step - huber_fn: 0.1980 - loss: 0.4029


In [82]:
precision = keras.metrics.Precision()

In [83]:
precision([0,1,1,1,0,1,0,1],[1,1,0,1,0,1,0,1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [84]:
precision([0,1,0,0,1,0,1,1],[1,0,1,1,0,0,0,0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [85]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [86]:
precision.variables

[<KerasVariable shape=(1,), dtype=float32, path=precision_1/true_positives>,
 <KerasVariable shape=(1,), dtype=float32, path=precision_1/false_positives>]

In [87]:
precision.reset_state()

In [88]:
class HuberMetrics(keras.metrics.Metric):
    def __init__(self,threshold=2.0,**kwargs):
        super().__init__(**kwargs) # handles base args(e.g. dtype )
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight(name="total",initializer="zeros")
        self.count = self.add_weight(name="count",initializer="zeros")
    def update_state(self,y_true,y_pred,sample_weight=None):
        metric = self.huber_fn(y_true,y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true),tf.float32))
    def result(self):
        return self.total/self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config,"threshold":self.threshold}
        

In [89]:
m = HuberMetrics(2.0)
m(tf.constant([[2.0]]),tf.constant([[10.]]))
# how done it let's see
#total  = 2*|10-2|-2^2/2
# count =1
# result 14/1=14

<tf.Tensor: shape=(), dtype=float32, numpy=14.0>

In [90]:
# total = total + (|1 - 0|² / 2) + (2 * |9.25 - 5| - 2² / 2) = 14 + 7 = 21
# count = count + 2 = 3
# result = total / count = 21 / 3 = 7
m(tf.constant([[0.], [5.]]), tf.constant([[1.], [9.25]]))

<tf.Tensor: shape=(), dtype=float32, numpy=7.0>

In [91]:
m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=7.0>

In [92]:
m.variables

[<KerasVariable shape=(), dtype=float32, path=huber_metrics_2/total>,
 <KerasVariable shape=(), dtype=float32, path=huber_metrics_2/count>]

In [93]:
m.reset_state()
m.variables

[<KerasVariable shape=(), dtype=float32, path=huber_metrics_2/total>,
 <KerasVariable shape=(), dtype=float32, path=huber_metrics_2/count>]

Lets check our HuberMetrics working or not 

In [94]:
tf.keras.utils.set_random_seed(42)
model = keras.models.Sequential([
    tf.keras.layers.Dense(30,activation="relu",kernel_initializer="he_normal",input_shape=input_shape),
    tf.keras.layers.Dense(1)
])

In [95]:
model.compile(loss=create_huber(2.0),optimizer="nadam",metrics=[HuberMetrics(2.0)])

In [96]:
model.fit(X_train_scaled,y_train,epochs=10,validation_data=(X_valid_scaled,y_valid))

Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 678us/step - huber_metrics_3: 1.0512 - loss: 1.0512 - val_huber_metrics_3: 0.5086 - val_loss: 0.5086
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - huber_metrics_3: 0.3174 - loss: 0.3174 - val_huber_metrics_3: 0.3527 - val_loss: 0.3527
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - huber_metrics_3: 0.2612 - loss: 0.2612 - val_huber_metrics_3: 0.2689 - val_loss: 0.2689
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 418us/step - huber_metrics_3: 0.2328 - loss: 0.2328 - val_huber_metrics_3: 0.2207 - val_loss: 0.2207
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - huber_metrics_3: 0.2178 - loss: 0.2178 - val_huber_metrics_3: 0.1899 - val_loss: 0.1899
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 418us/step - huber_metrics_3: 0.2093 - loss: 0.2093 - val_huber_metrics_3: 0.1996 - val_loss: 0.1996
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - huber_metrics_3: 0.2036 - loss: 0.2036 - val_huber_metrics_3: 0.2243 - val_loss:

In [97]:
model.save("custom-sequanlial-model-with-custom-metrics.keras")

In [98]:
model=tf.keras.models.load_model("./custom-sequanlial-model-with-custom-metrics.keras",custom_objects={"huber_fn":huber_fn,
                                                                                                       "HuberMetrics":HuberMetrics})

In [99]:
model.fit(X_train_scaled,y_train,epochs=3,validation_data=(X_valid_scaled,y_valid))

Epoch 1/3
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step - huber_metrics_3: 1.6087 - loss: 1.0373 - val_huber_metrics_3: 2.2222 - val_loss: 0.7968
Epoch 2/3
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 418us/step - huber_metrics_3: 2.3350 - loss: 0.8245 - val_huber_metrics_3: 2.2330 - val_loss: 0.7840
Epoch 3/3
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 420us/step - huber_metrics_3: 2.3522 - loss: 0.8146 - val_huber_metrics_3: 2.2478 - val_loss: 0.7787


make it more reliable to model 

In [100]:
class HuberMetrics(tf.keras.metrics.Mean):
    def __init__(self, threshold=2.0,name="Hubermetrics",dtype=None):
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        super().__init__(name=name,dtype=dtype)
    def update_state(self,y_true,y_pred,sample_weight=None):
        metric= self.huber_fn(y_true,y_pred)
        super(HuberMetrics,self).update_state(metric,sample_weight)
    def get_config(self):
        base_config= super().get_config()
        return {**base_config,"threshold":self.threshold}
        

In [101]:
tf.keras.utils.set_random_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal",
                          input_shape=input_shape),
    tf.keras.layers.Dense(1),
])

In [102]:
model.compile(loss=tf.keras.losses.Huber(2.0), optimizer="nadam",
              weighted_metrics=[HuberMetrics(2.0)])

In [103]:
sample_weight = np.random.rand(len(y_train))
history = model.fit(X_train_scaled, y_train, epochs=2,
                    sample_weight=sample_weight)

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 355us/step - Hubermetrics: 1.0599 - loss: 0.5274
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 356us/step - Hubermetrics: 0.3215 - loss: 0.1598


In [104]:
(history.history["loss"][0],
history.history["Hubermetrics"][0] * sample_weight.mean())

(0.3256884217262268, 0.32568849524955656)

In [105]:
model.save("my_model_with_a_custom_metric_v2.keras")

In [106]:
model = tf.keras.models.load_model("my_model_with_a_custom_metric_v2.keras",
                                   custom_objects={"HuberMetrics": HuberMetrics})

In [107]:
model.fit(X_train_scaled, y_train, epochs=2)

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 323us/step - Hubermetrics: 0.2626 - loss: 0.2261
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 316us/step - Hubermetrics: 0.2298 - loss: 0.1999


Custom Layers

In [108]:
exponential_layer = keras.layers.Lambda(lambda x:tf.exp(x))

In [109]:
exponential_layer(tf.constant([1.,2.,3.]))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 2.7182817,  7.389056 , 20.085537 ], dtype=float32)>

In [110]:
## adding the exponential layer on model 
tf.keras.utils.set_random_seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(30,activation="relu",kernel_initializer="he_normal",input_shape=input_shape),
    tf.keras.layers.Dense(1),
    exponential_layer
])
model.compile(optimizer="sgd",loss="mse")
model.fit(X_train_scaled,y_train,epochs=10,validation_data=(X_valid_scaled,y_valid))
model.evaluate(X_test_scaled,y_test)

Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 559us/step - loss: 6.2862 - val_loss: 5.4560
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step - loss: 5.7819 - val_loss: 5.4560
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 361us/step - loss: 5.7819 - val_loss: 5.4560
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step - loss: 5.7819 - val_loss: 5.4560
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 396us/step - loss: 5.7819 - val_loss: 5.4560
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 5.7819 - val_loss: 5.4560
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step - loss: 5.7823 - val_loss: 5.4560
Epoch 8/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step - loss: 5.7819 - val_loss: 5.4560
Epoch 9/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step - loss: 5.7842 - val_loss: 5.4560
Epoch 10/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step - loss: 5.7819 - val_loss: 5.4560
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 218us/step - loss: 5.5903


5.579812526702881

In [111]:
class MyDense(keras.layers.Layer):
    def __init__(self,units,activation=None,**kwargs):
        super().__init__(**kwargs)
        self.units= units
        self.activation = keras.activations.get(activation)
        
    def build(self,batch_input_shape):# Create the layer's variable by calling add_weight()
        self.kernel=self.add_weight(
            name='kernal',shape=[batch_input_shape[-1],self.units],
            initializer = 'he_normal')
        self.bias = self.add_weight(
            name='bias',shape=[self.units],initializer = 'zeros')
        super().build(batch_input_shape)
        
    def call(self,X): # Perform the desired operations . In this case ,we compute the metrix mutiplication of the inputs X and the layer's kernel 
        return self.activation(X @ self.kernel+self.bias)
    
    def get_config(self): 
        base_config=super().get_config()
        return {**base_config,"units":self.units,
                "activation":keras.activation.serializer(self.activation)}

In [112]:
# lets create the model with custom model 
tf.keras.utils.set_random_seed(42)
model = tf.keras.Sequential([
    MyDense(30,activation="relu",input_shape=input_shape),
    MyDense(1)
    
])
model.compile(optimizer="nadam",loss="mse")
model.fit(X_train_scaled,y_train,validation_data=(X_valid_scaled,y_valid),epochs=10)
model.evaluate(X_test_scaled,y_test)

Epoch 1/10


/var/folders/7z/kfcjh8xj1wbc9dktydd14w540000gn/T/ipykernel_1591/4222156707.py:3: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 533us/step - loss: 5.7265 - val_loss: 6.9255
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 389us/step - loss: 0.9550 - val_loss: 2.6011
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 385us/step - loss: 0.7034 - val_loss: 0.8613
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step - loss: 0.5803 - val_loss: 0.4588
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 385us/step - loss: 0.5049 - val_loss: 0.4805
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 385us/step - loss: 0.4642 - val_loss: 0.4709
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 465us/step - loss: 0.4396 - val_loss: 0.4394
Epoch 8/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 396us/step - loss: 0.4249 - val_loss: 0.3929
Epoch 9/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 375us/step - loss: 0.4142 - val_loss: 0.4076
Epoch 10/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 364us/step - loss: 0.4059 - val_loss: 0.3804
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 207us/step - loss: 0.3991


0.3931061625480652

In [113]:
class MyMultiLayer(keras.layers.Layer):
    def call(self,X):
        X1,X2=X
        print("X1.shape: ", X1.shape ," X2.shape: ", X2.shape)
        return [X1+X2,X1*X2,X1/X2]
    
    def compute_output_shape(self,batch_input_shape):
        b1,b2 = batch_input_shape
        return [b1,b1,b1] # should probably handle broadcasting rules 

In [114]:
#test the multilayer 
input1= tf.keras.Input(shape=[2])
input2= tf.keras.Input(shape=[2])
MyMultiLayer()((input1,input2))

[<KerasTensor shape=(None, 2), dtype=float32, sparse=False, name=keras_tensor_68>,
 <KerasTensor shape=(None, 2), dtype=float32, sparse=False, name=keras_tensor_69>,
 <KerasTensor shape=(None, 2), dtype=float32, sparse=False, name=keras_tensor_70>]

In [115]:
# extra code – tests MyMultiLayer with actual data 
X1, X2 = np.array([[3., 6.], [2., 7.]]), np.array([[6., 12.], [4., 3.]]) 
MyMultiLayer()((X1, X2))

X1.shape:  (2, 2)  X2.shape:  (2, 2)


[<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[ 9., 18.],
        [ 6., 10.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[18., 72.],
        [ 8., 21.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.5      , 0.5      ],
        [0.5      , 2.3333333]], dtype=float32)>]

In [116]:
# Building the layer with different behaviour during training and during testing 
class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__( **kwargs)
        self.stddev= stddev
    
    def call(self,X,training=None):
        if training:
            noise=tf.random.normal(tf.shape(X),stddev=self.stddev)
            return X+noise
        else:
            return X
    
    def compute_output_shape(self,batch_input_shape):
        return batch_input_shape

In [117]:
tf.keras.utils.set_random_seed(42)
model = tf.keras.Sequential([
    MyGaussianNoise(stddev=1.0, input_shape=input_shape),
    tf.keras.layers.Dense(30, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1)
])
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Epoch 1/2


/var/folders/7z/kfcjh8xj1wbc9dktydd14w540000gn/T/ipykernel_1591/2361834058.py:4: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__( **kwargs)


363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 808us/step - loss: 2.7619 - val_loss: 25.1369
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 394us/step - loss: 1.3951 - val_loss: 14.9793
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 234us/step - loss: 1.1206


1.1244221925735474

custom models

In [118]:
# Residualblock layer 
class Residualblock(keras.layers.Layer):
    def __init__(self,n_layers , n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(n_neurons,activation="elu",kernel_initializer="he_normal")
                       for _ in range(n_layers)]
    def call(self,inputs):
        Z=inputs
        for layer in self.hidden:
            Z=layer(Z)
        return inputs+Z

In [119]:
class ResidualRegressor(keras.models.Model):
    def __init__(self,output_dim,**kwargs):
        super().__init__(**kwargs)
        self.output_dim = output_dim
        self.hidden1 = keras.layers.Dense(30,activation="elu",kernel_initializer="he_normal")
        self.block1 = Residualblock(2,30)
        self.block2 = Residualblock(2,30)
        self.out=keras.layers.Dense(output_dim)
        
    def call(self,inputs):
        Z = self.hidden1(inputs)
        for _ in range(1+3):
            Z= self.block1(Z)
        return self.out(Z)
    
    def get_config(self): # for: that able to load and save model
        base_config = super().get_config()
        return {**base_config,"output_dim":self.output_dim}

In [120]:
# extra code – shows that the model can be used normally
tf.keras.utils.set_random_seed(42)
model = ResidualRegressor(1)
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=2)
score = model.evaluate(X_test_scaled, y_test)
model.save("my_custom_model.keras")

Epoch 1/2


/opt/anaconda3/envs/Data_pytorcch/lib/python3.11/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'residual_regressor', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 522us/step - loss: 93.5322 
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step - loss: 2.2380
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 328us/step - loss: 1.4496


In [121]:
# extra code – the model can be loaded and you can continue training or use it
#              to make predictions
model = tf.keras.models.load_model(
    "my_custom_model.keras",
    custom_objects={"ResidualRegressor": ResidualRegressor}
)
history = model.fit(X_train_scaled, y_train, epochs=2)
model.predict(X_test_scaled[:3])

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 527us/step - loss: 1.2329
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 613us/step - loss: 0.9327
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


array([[1.1215348],
       [2.0058653],
       [3.7029212]], dtype=float32)

We could have defined the model using the sequential API instead:


In [122]:
tf.keras.utils.set_random_seed(42)
block1 = Residualblock(2, 30)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation="relu",
                          kernel_initializer="he_normal"),
    block1, block1, block1, block1,
    Residualblock(2, 30),
    tf.keras.layers.Dense(1)
])

Losses and Metrics Based on Model Internals

In [123]:
class ReconstructingRegresser(tf.keras.Model):
    def __init__(self,output_dim,**kwargs):
        super().__init__(**kwargs)
        self.hidden=[keras.layers.Dense(30,activation="relu",
                                        kernel_initializer="he_normal")
                     for _ in range(5)]
        self.out =tf.keras.layers.Dense(output_dim)
        
    def build(self,batch_input_shape):
        n_inputs=batch_input_shape[-1]
        self.reconstruct =tf.keras.layers.Dense(n_inputs)
    
    def call(self,inputs,training=None):
        Z= inputs 
        for layer in self.hidden:
            Z = layer(Z)
        recontruction=self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(recontruction-inputs))
        self.add_loss(0.05*recon_loss)
        return self.out(Z)
    

In [124]:
# extra code
tf.keras.utils.set_random_seed(42)
model = ReconstructingRegresser(1)
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=5)
y_pred = model.predict(X_test_scaled)

Epoch 1/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 555us/step - loss: 1.1051
Epoch 2/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step - loss: 0.5082
Epoch 3/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 682us/step - loss: 0.4392
Epoch 4/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step - loss: 0.3985
Epoch 5/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 518us/step - loss: 0.3805
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step


Computing Gradients Using AutoDiff 

In [125]:
def f(w1,w2):
    return 3 *w1**2+2*w1*w2

In [126]:
f(3,6)

63

In [127]:
w1,w2 = 5,3
eps = 1e-6
(f(w1+eps,w2)-f(w1,w2))/eps

36.000003007075065

In [128]:
(f(w1,w2+eps)-f(w1,w2))/eps

10.000000003174137

In [129]:
# using autodiff
w1,w2= tf.Variable(5.),tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1,w2)
gradients = tape.gradient(z,[w1,w2])

In [130]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [134]:
with tf.GradientTape(persistent=True) as tape:
    z= f(w1,w2)
dz_dw1=tape.gradient(z,w1)
dz_dw2=tape.gradient(z,w2)
del tape


In [135]:
dz_dw1,dz_dw2

(<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>)

In [137]:
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    z = f(c1, c2)

gradients = tape.gradient(z, [c1, c2])

In [138]:
gradients

[None, None]

In [139]:
with tf.GradientTape() as tape:
    tape.watch(c1)
    tape.watch(c2)
    z=f(c1,c2)
gradients = tape.gradient(z,[c1,c2])

In [140]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [142]:
with tf.GradientTape(persistent=True) as hessian_tape:
    with tf.GradientTape() as jacobian_tape:
        z= f(w1,w2)
    jacobians= jacobian_tape.gradient(z,[w1,w2])
hessians = [hessian_tape.gradient(jacobians,[w1,w2])
            for jacobian in jacobians]
del hessian_tape

In [143]:
hessians

[[<tf.Tensor: shape=(), dtype=float32, numpy=8.0>,
  <tf.Tensor: shape=(), dtype=float32, numpy=2.0>],
 [<tf.Tensor: shape=(), dtype=float32, numpy=8.0>,
  <tf.Tensor: shape=(), dtype=float32, numpy=2.0>]]

In [144]:
jacobians

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [145]:
def f(w1,w2):
    return 3*w1**2+tf.stop_gradient(2*w1*w2)

with tf.GradientTape() as tape:
    z = f(w1,w2)
gradients = tape.gradient(z,[w1,w2])

In [146]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=30.0>, None]

In [147]:
x = tf.Variable([100.])
with tf.GradientTape() as tape:
    z = tf.sqrt(x)
gradients = tape.gradient(z,[x])

In [148]:
gradients

[<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.05], dtype=float32)>]

In [153]:
x = tf.Variable([1.0e30])
with tf.GradientTape() as tape:
    z = my_softpluz(x)
    
tape.gradient(z,[x])

[<tf.Tensor: shape=(1,), dtype=float32, numpy=array([nan], dtype=float32)>]

In [154]:
def my_softplus(z):
    return tf.math.log(1 + tf.exp(-tf.abs(z))) + tf.maximum(0., z)

Here is the proof that this equation is equal to log(1 + exp(z)):

softplus(z) = log(1 + exp(z))

softplus(z) = log(1 + exp(z)) - log(exp(z)) + log(exp(z)) ; just adding and subtracting the same value

softplus(z) = log[(1 + exp(z)) / exp(z)] + log(exp(z)) ; since log(a) - log(b) = log(a / b)

softplus(z) = log[(1 + exp(z)) / exp(z)] + z ; since log(exp(z)) = z

softplus(z) = log[1 / exp(z) + exp(z) / exp(z)] + z ; since (1 + a) / b = 1 / b + a / b

softplus(z) = log[exp(–z) + 1] + z ; since 1 / exp(z) = exp(–z), and exp(z) / exp(z) = 1

softplus(z) = softplus(–z) + z ; we recognize the definition at the top, but with –z

softplus(z) = softplus(–|z|) + max(0, z) ; if you consider both cases, z < 0 or z ≥ 0, you will see that this works

In [155]:
@tf.custom_gradient
def my_better_softplus(z):
    exp = tf.exp(z)
    def my_softplus_gradient(grad):
        return grad/(1+1/exp)
    return tf.math.log(exp+1),my_softplus_gradient

In [156]:
# extra code – shows that the function is now stable, as well as its gradients
x = tf.Variable([1000.])
with tf.GradientTape() as tape:
    z = my_softplus(x)

z, tape.gradient(z, [x])

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1000.], dtype=float32)>,
 [<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>])

Custom Training Loop

In [157]:
tf.keras.utils.set_random_seed(42)
l2_reg = keras.regularizers.l2(0.05)
model = keras.models.Sequential([
    keras.layers.Dense(30,activation='relu',kernel_initializer="he_normal",kernel_regularizer=l2_reg),
    keras.layers.Dense(1,kernel_regularizer=l2_reg)
])

In [159]:
def random_batch(X,y, batch_size=32):
    idx = np.random.randint(len(X),size=batch_size)
    return X[idx] , y[idx]

In [160]:
def print_status_bar(iteration,total,loss,metrics = None):
    metrics = " - ".join(["{}:{:.4f}".format(m.name,m.result())
                          for m in [loss]+(metrics or [])])
    end = " " if iteration<total<total else "\n"
    print("\r{}/{} - ".format(iteration,total)+metrics,end=end)

In [162]:
n_epochs =5
batch_size = 32
n_steps = len(X_train)//batch_size
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.MeanSquaredError()
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]

There’s a lot going on in this code, so let’s walk through it:
• We create two nested loops: one for the epochs, the other for the batches within
an epoch.

• Then we sample a random batch from the training set.

• Inside the tf.GradientTape() block, we make a prediction for one batch (using
the model as a function), and we compute the loss: it is equal to the main loss
plus the other losses (in this model, there is one regularization loss per layer).

Since the mean_squared_error() function returns one loss per instance, we
compute the mean over the batch using tf.reduce_mean() (if you wanted to
apply different weights to each instance, this is where you would do it). The regu‐
larization losses are already reduced to a single scalar each, so we just need to
sum them (using tf.add_n(), which sums multiple tensors of the same shape
and data type).

• Next, we ask the tape to compute the gradient of the loss with regards to each
trainable variable (not all variables!), and we apply them to the optimizer to per‐
form a Gradient Descent step.

• Next we update the mean loss and the metrics (over the current epoch), and we
display the status bar.

• At the end of each epoch, we display the status bar again to make it look com‐
plete11 and to print a line feed, and we reset the states of the mean loss and the
metrics.

In [164]:
for epoch in range(1,n_epochs+1):
    print("Epoch{}/{}".format(epoch,n_epochs))
    for step in range(1,n_steps+1):
        X_batch,y_batch = random_batch(X_train_scaled,y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch,training = True)
            main_loss = tf.reduce_mean(loss_fn(y_batch,y_pred))
            loss = tf.add_n([main_loss]+model.losses)
        gradients= tape.gradient(loss,model.trainable_variables)
        optimizer.apply_gradients(zip(gradients,model.trainable_variables))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch,y_pred)
        print_status_bar(step*batch_size,len(y_train),mean_loss,metrics)
    print_status_bar(len(y_train),len(y_train),mean_loss,metrics)
    for metric in [mean_loss]+metrics:
        metric.reset_states()

Epoch1/5
32/11610 - mean:9.8421 - mean_absolute_error:1.8926
64/11610 - mean:8.9422 - mean_absolute_error:1.8288
96/11610 - mean:8.6537 - mean_absolute_error:1.7784
128/11610 - mean:8.4475 - mean_absolute_error:1.7439
160/11610 - mean:7.9601 - mean_absolute_error:1.6643
192/11610 - mean:7.5523 - mean_absolute_error:1.5714
224/11610 - mean:7.5341 - mean_absolute_error:1.5677
256/11610 - mean:7.3369 - mean_absolute_error:1.5274
288/11610 - mean:7.1592 - mean_absolute_error:1.4853
320/11610 - mean:7.0650 - mean_absolute_error:1.4684
352/11610 - mean:6.8702 - mean_absolute_error:1.4234
384/11610 - mean:6.6834 - mean_absolute_error:1.3767
416/11610 - mean:6.5661 - mean_absolute_error:1.3502
448/11610 - mean:6.4240 - mean_absolute_error:1.3234
480/11610 - mean:6.3097 - mean_absolute_error:1.2977
512/11610 - mean:6.1904 - mean_absolute_error:1.2737
544/11610 - mean:6.0755 - mean_absolute_error:1.2513
576/11610 - mean:5.9891 - mean_absolute_error:1.2390
608/11610 - mean:5.8657 - mean_absolute_

AttributeError: 'Mean' object has no attribute 'reset_states'

In [165]:
for epoch in range(1, n_epochs + 1):
    print(f"Epoch {epoch}/{n_epochs}")
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # extra code – if your model has variable constraints
        for variable in model.variables:
            if variable.constraint is not None:
                variable.assign(variable.constraint(variable))

        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)

        print_status_bar(step, n_steps, mean_loss, metrics)

    for metric in [mean_loss] + metrics:
        metric.reset_state()

Epoch 1/5
1/362 - mean:1.3766 - mean_absolute_error:0.5496
2/362 - mean:1.3741 - mean_absolute_error:0.5493
3/362 - mean:1.3718 - mean_absolute_error:0.5490
4/362 - mean:1.3694 - mean_absolute_error:0.5488
5/362 - mean:1.3688 - mean_absolute_error:0.5492
6/362 - mean:1.3679 - mean_absolute_error:0.5494
7/362 - mean:1.3659 - mean_absolute_error:0.5492
8/362 - mean:1.3634 - mean_absolute_error:0.5488
9/362 - mean:1.3611 - mean_absolute_error:0.5485
10/362 - mean:1.3589 - mean_absolute_error:0.5484
11/362 - mean:1.3573 - mean_absolute_error:0.5485
12/362 - mean:1.3551 - mean_absolute_error:0.5482
13/362 - mean:1.3529 - mean_absolute_error:0.5480
14/362 - mean:1.3517 - mean_absolute_error:0.5481
15/362 - mean:1.3495 - mean_absolute_error:0.5480
16/362 - mean:1.3475 - mean_absolute_error:0.5477
17/362 - mean:1.3465 - mean_absolute_error:0.5478
18/362 - mean:1.3459 - mean_absolute_error:0.5481
19/362 - mean:1.3454 - mean_absolute_error:0.5484
20/362 - mean:1.3436 - mean_absolute_error:0.5484

Tensorflow Functions and Graphs

In [167]:
def cube(x):
    return x**3
cube(3)

27

In [168]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [169]:
tf_cube = tf.function(cube)
tf_cube

In [170]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [171]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [172]:
@tf.function
def tf_cube(x):
    return x**3

In [173]:
tf_cube.python_function(2)

8

Tensorflow functions and concrete functions 

In [174]:
concrete_function = tf_cube.get_concrete_function(tf.constant(2.0))
concrete_function

<ConcreteFunction (x: TensorSpec(shape=(), dtype=tf.float32, name=None)) -> TensorSpec(shape=(), dtype=tf.float32, name=None) at 0x306E5A6D0>

In [175]:
concrete_function(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [176]:
concrete_function is tf_cube.get_concrete_function(tf.constant(2.0))

True

Exploring Functions Definiations of Graphs

In [177]:
concrete_function.graph

In [178]:
ops = concrete_function.graph.get_operations()
ops

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'pow/y' type=Const>,
 <tf.Operation 'pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [179]:
pow_op = ops[2]
list(pow_op.inputs)

[<tf.Tensor 'x:0' shape=() dtype=float32>,
 <tf.Tensor 'pow/y:0' shape=() dtype=float32>]